## 1. Quadratic Divergence

#### 1.1 Basic problem

Find a pair $(\mu,v)$ that satisfy
\begin{equation}
\mu = \min_{N_1\in \mathcal{N}}\mathbb E \left(N_1 \left[g(X_1)+v_1\right]+\frac{\xi}{2}(N_1^2-N_1) \mid \mathfrak{I}_0\right) - v_0
\end{equation}

subject to:
\begin{align*}
&\mathbb{E}\left[N_1f(X_1)\mid\mathfrak{I}_0\right] = 0\\
&\mathbb{E}\left[N_1\mid\mathfrak{I}_0\right] = 1
\end{align*}

#### 1.2 Dual problem
\begin{equation}
\mu = \max_{\lambda_1,\lambda_2} -\frac{\xi}{2}\mathbb{E}\left[\left(\left[\frac{1}{2}-\frac{1}{\xi}\left[g(X)+v_1+\lambda_1 \cdot f(X) + \lambda_2\right]\right]^+\right)^2\mid\mathfrak{I}_0\right]-\lambda_2-v_0
\end{equation}

The implied solution for the probablity distortion is:

\begin{equation}
N_1^* = \left[\frac{1}{2} - \frac{1}{\xi^*}\left[g(X)+v_1^*+\lambda_1^*\cdot f(X)+\lambda_2^*\right]\right]^+
\end{equation}

## 2. Code Implementation


In [2]:
# Load packages
import time
from utilities_quadratic import *
from plotting_module_quadratic import *

### 3.1 Data
The file “UnitaryData.csv” contains the following variables:

- The first four columns contain Euler equation errors from the unitary risk aversion model corresponding to the 3-month T-bill rate, the market excess return, the SMB excess return, and the HML excess return respectively. Under a feasible belief distortion, all four of these variables should have expectation zero (conditional or unconditional).


- The column “d.p” contains the dividend-price ratio for the CRSP value weighted index, computed at the start of the return period. Hence functions of d.p[i] (i.e. quantile indicator functions) are valid instruments for the returns in row i.


- The final column “log.RW” contains values of the logarithmic return on wealth. This is the random variable who’s expectation we are intersted in bounding.

All returns are quarterly and inflation adjusted.

In [17]:
# Load data
data = pd.read_csv('UnitaryData.csv')
# Show statistics of the data
data.describe()

,Rf,Rm-Rf,SMB,HML,d.p,log.RW
count,248.000000,248.000000,248.000000,248.000000,248.000000,248.000000
mean,-0.010974,0.010974,0.004992,0.009498,0.029250,0.018864
std,0.086116,0.086116,0.052512,0.058218,0.010055,0.084452
min,-0.186245,-0.356102,-0.139624,-0.269700,0.010571,-0.312496
25%,-0.063218,-0.029274,-0.026004,-0.024469,0.020545,-0.021638
50%,-0.028582,0.028582,0.000834,0.008563,0.029106,0.031677
75%,0.029274,0.063218,0.035165,0.036895,0.036495,0.069903
max,0.356102,0.186245,0.169266,0.234932,0.058402,0.210429


We first compute the terciles of dividend-price ratio and form indicator functions of these terciles. Then we multiply the first four columns by the indicator functions to form a 12-dimensional $f$. The last column log.RW is our $g$.

### 3.2 Results

In [18]:
# Count time
time_start = time.time() 

# Initialize the solver
solver = InterDivConstraint(tol=1e-9,max_iter=1000)

# Define g(X) = log Rw
solver.g = solver.log_Rw

# Find ξ that corresponds to 120% min QD
x_min_QD = 1.2
# ξ_lower = solver.find_ξ(x_min_QD=x_min_QD,lower=True,tol=1e-7,max_iter=100)
# ξ_upper = solver.find_ξ(x_min_QD=x_min_QD,lower=False,tol=1e-7,max_iter=100)
ξ_lower = 0.1672520637512207
ξ_upper = 0.15077531337738037

# Solve models with the chosen ξ
result_lower = solver.iterate(ξ_lower,lower=True)
result_upper = solver.iterate(ξ_upper,lower=False)

# Print out the time spent
time_spent = round(time.time()-time_start,4)

In [19]:
# result_min = solver.iterate(10.,lower=False)

# N = result_min['N']
# np.sum(np.absolute(N) < 1e-5)

# # Load data
# data = pd.read_csv('UnitaryData.csv')
# # Show statistics of the data
# data.describe()

# solver.g[N==0]

# np.sum(solver.X[:,3]>0.11082118)/247

# solver.X[N==0]

# solver.pd_lag_indicator[N==0]

In [20]:
# Print iteration information
print("--- Iteration Ends ---")
print("%s percent min quadratic divergence" % (int(x_min_QD*100)))
print("Time spent: %s seconds ---" % (time_spent))

# Print converged parameter results
print("\n")
print("--- Converged values for the lower bound problem ---")
print("λ_1: %s" % result_lower['λ_1'])
print("λ_2: %s" % result_lower['λ_2'])
print("μ: %s" % result_lower['μ'])
print("v: %s" % result_lower['v'])

print(" ")
print("--- Converged values for the upper bound problem ---")
print("λ_1: %s" % result_lower['λ_1'])
print("λ_2: %s" % result_lower['λ_2'])
print("μ: %s" % result_lower['μ'])
print("v: %s" % result_lower['v'])

# Print transition probability matrix under the original empirical probability
print("\n")
print("--- Transition Probability Matrix (Original) ---")
print(result_lower['P'])

# Print transition probability matrix under distorted probability, lower bound
print(" ")
print("--- Transition Probability Matrix (Distorted, lower bound problem) ---")
print(result_lower['P_tilde'])

# Print transition probability matrix under distorted probability, upper bound
print(" ")
print("--- Transition Probability Matrix (Distorted, upper bound problem) ---")
print(result_upper['P_tilde'])


# Print stationary distribution under the original empirical probability
print("\n")
print("--- Stationary Distribution (Original) ---")
print(result_lower['π'])

# Print stationary distribution under distorted probability, lower bound
print(" ")
print("--- Stationary Distribution (Distorted, lower bound problem) ---")
print(result_lower['π_tilde'])

# Print stationary distribution under distorted probability, upper bound
print(" ")
print("--- Stationary Distribution (Distorted, upper bound problem) ---")
print(result_upper['π_tilde'])

# Print quadratic divergence
print("\n")
print("--- Quadratic Divergence (lower bound problem) ---")
print("1/2*E[N^2-N|state 1] = %s " % result_lower['QD_cond'][0])
print("1/2*E[N^2-N|state 2] = %s " % result_lower['QD_cond'][1])
print("1/2*E[N^2-N|state 3] = %s " % result_lower['QD_cond'][2])
print("1/2*E[N^2-N]         = %s " % result_lower['QD'])

# Print quadratic divergence
print(" ")
print("--- Quadratic Divergence (Upper bound problem) ---")
print("1/2*E[N^2-N|state 1] = %s " % result_upper['QD_cond'][0])
print("1/2*E[N^2-N|state 2] = %s " % result_upper['QD_cond'][1])
print("1/2*E[N^2-N|state 3] = %s " % result_upper['QD_cond'][2])
print("1/2*E[N^2-N]         = %s " % result_upper['QD'])

# Print conditional moment & bounds
print("\n")
print("--- Moment (Original, annualized, %) ---")
print("E[g(X)|state 1]  = %s " % (result_lower['moment_cond'][0]*400))
print("E[g(X)|state 2]  = %s " % (result_lower['moment_cond'][1]*400))
print("E[g(X)|state 3]  = %s " % (result_lower['moment_cond'][2]*400))
print("E[g(X)]  = %s " % (result_lower['moment']*400))
print(" ")
print("--- Moment (Lower bound, annualized, %) ---")
print("E[Ng(X)|state 1] = %s " % (result_lower['moment_bound_cond'][0]*400))
print("E[Ng(X)|state 2] = %s " % (result_lower['moment_bound_cond'][1]*400))
print("E[Ng(X)|state 3] = %s " % (result_lower['moment_bound_cond'][2]*400))
print("E[Ng(X)] = %s " % (result_lower['moment_bound']*400))
print(" ")
print("--- Moment (Upper bound, annualized, %) ---")
print("E[Ng(X)|state 1] = %s " % (result_upper['moment_bound_cond'][0]*400))
print("E[Ng(X)|state 2] = %s " % (result_upper['moment_bound_cond'][1]*400))
print("E[Ng(X)|state 3] = %s " % (result_upper['moment_bound_cond'][2]*400))
print("E[Ng(X)] = %s " % (result_upper['moment_bound']*400))

--- Iteration Ends ---
120 percent min quadratic divergence
Time spent: 2.8255 seconds ---


--- Converged values for the lower bound problem ---
λ_1: [ 1.34627142  0.65372767  0.06601434  0.16296256  1.29835833  0.70163979
 -0.09703071  1.00191807  1.10832483  0.891676    0.71850002  1.38196166]
λ_2: [-0.09761062 -0.26405642 -0.4639388 ]
μ: 0.009577535386226951
v: [0.         0.15696621 0.3414455 ]
 
--- Converged values for the upper bound problem ---
λ_1: [ 1.34627142  0.65372767  0.06601434  0.16296256  1.29835833  0.70163979
 -0.09703071  1.00191807  1.10832483  0.891676    0.71850002  1.38196166]
λ_2: [-0.09761062 -0.26405642 -0.4639388 ]
μ: 0.009577535386226951
v: [0.         0.15696621 0.3414455 ]


--- Transition Probability Matrix (Original) ---
[[0.96341463 0.03658537 0.        ]
 [0.04878049 0.87804878 0.07317073]
 [0.         0.08433735 0.91566265]]
 
--- Transition Probability Matrix (Distorted, lower bound problem) ---
[[0.99146397 0.00853569 0.        ]
 [0.08332584 0.9

In [ ]:
time_start = time.time() 
entropy_moment_bounds()
print("Time spent: %s seconds ---" % (round(time.time()-time_start,4)))

## Tables and plots

In [13]:
# Use 20% higher than min QD
x_min_QD = 1.2

# Case 1: g(X) = log Rw
solver_1 = InterDivConstraint(tol=1e-9,max_iter=1000)
solver_1.g = solver_1.log_Rw
# ξ_1_lower = solver_1.find_ξ(x_min_QD=x_min_QD,lower=True,tol=1e-7,max_iter=100)
# ξ_1_upper = solver_1.find_ξ(x_min_QD=x_min_QD,lower=False,tol=1e-7,max_iter=100)
ξ_1_lower = 0.1672520637512207
ξ_1_upper = 0.15077531337738037
result_1_lower = solver_1.iterate(ξ_1_lower,lower=True)
result_1_upper = solver_1.iterate(ξ_1_upper,lower=False)
result_1_min = solver_1.iterate(10.,lower=True)

# Case 2: g(X) = Rw
solver_2 = InterDivConstraint(tol=1e-9,max_iter=1000)
solver_2.g = np.exp(solver_2.log_Rw)
# ξ_2_lower = solver_2.find_ξ(x_min_QD=x_min_QD,lower=True,tol=1e-7,max_iter=100)
# ξ_2_upper = solver_2.find_ξ(x_min_QD=x_min_QD,lower=False,tol=1e-7,max_iter=100)
ξ_2_lower = 0.18506479263305664
ξ_2_upper = 0.17623425461351871
result_2_lower = solver_2.iterate(ξ_2_lower,lower=True)
result_2_upper = solver_2.iterate(ξ_2_upper,lower=False)
result_2_min = solver_2.iterate(10.,lower=True)

# Case 3: g(X) = Rw/Rf
solver_3 = InterDivConstraint(tol=1e-9,max_iter=1000)
solver_3.g = 1./(solver_3.X[:,0]+1.)
# ξ_3_lower = solver_3.find_ξ(x_min_QD=x_min_QD,lower=True,tol=1e-7,max_iter=100)
# ξ_3_upper = solver_3.find_ξ(x_min_QD=x_min_QD,lower=False,tol=1e-7,max_iter=100)
ξ_3_lower = 0.0971330399622081
ξ_3_upper = 0.09415056183934212
result_3_lower = solver_3.iterate(ξ_3_lower,lower=True)
result_3_upper = solver_3.iterate(ξ_3_upper,lower=False)
result_3_min = solver_3.iterate(10.,lower=True)

In [14]:
print('Table 1: Probabilities')
print('')
print('Transition matrix and stationary distribution:')
print("")

print("--- empirical ---")
print('    ',np.round(result_1_min['P'][0],2))
print('P = ',np.round(result_1_min['P'][1],2))
print('    ',np.round(result_1_min['P'][2],2))
print('')
print('π = ',np.round(result_1_min['π'],2))
print('\n')

print("--- min quaratic divergence ---")
print('    ',np.round(result_1_min['P_tilde'][0],2))
print('P = ',np.round(result_1_min['P_tilde'][1],2))
print('    ',np.round(result_1_min['P_tilde'][2],2))
print('')
print('π = ',np.round(result_1_min['π_tilde'],2))
print('\n')

print("--- %s percent min quaratic divergence, lower bound ---" % (int(x_min_QD*100)))
print('    ',np.round(result_1_lower['P_tilde'][0],2))
print('P = ',np.round(result_1_lower['P_tilde'][1],2))
print('    ',np.round(result_1_lower['P_tilde'][2],2))
print('')
print('π = ',np.round(result_1_lower['π_tilde'],2))
print('\n')

print("--- %s percent min quaratic divergence, upper bound ---" % (int(x_min_QD*100)))
print('    ',np.round(result_1_upper['P_tilde'][0],2))
print('P = ',np.round(result_1_upper['P_tilde'][1],2))
print('    ',np.round(result_1_upper['P_tilde'][2],2))
print('')
print('π = ',np.round(result_1_upper['π_tilde'],2))

Table 1: Probabilities

Transition matrix and stationary distribution:

--- empirical ---
     [0.96 0.04 0.  ]
P =  [0.05 0.88 0.07]
     [0.   0.08 0.92]

π =  [0.42 0.31 0.27]


--- min quaratic divergence ---
     [0.98 0.02 0.  ]
P =  [0.07 0.9  0.02]
     [0.   0.15 0.85]

π =  [0.78 0.19 0.03]


--- 120 percent min quaratic divergence, lower bound ---
     [0.99 0.01 0.  ]
P =  [0.08 0.9  0.01]
     [0.   0.17 0.83]

π =  [0.9  0.09 0.01]


--- 120 percent min quaratic divergence, upper bound ---
     [0.97 0.03 0.  ]
P =  [0.07 0.9  0.03]
     [0.   0.14 0.86]

π =  [0.67 0.27 0.06]


In [15]:
print('Table 2: Expected market return')
print('--------------------------------------------------------------------')
print('conditioning    empirical    min divergence imputed    Rw/Rf imputed')
print('                 average         (lower, upper)        (lower,upper)')
print('--------------------------------------------------------------------')
print('low D/P           %s                %s                  %s' \
      % (np.round(np.log(result_2_min['moment_cond'][0])*400,2),np.round(np.log(result_2_min['moment_bound_cond'][0])*400,2),np.round(np.log(result_3_min['moment_bound_cond'][0])*400,2)))
print('                                  (%s,%s)           (%s,%s)' \
      % (np.round(np.log(result_2_lower['moment_bound_cond'][0])*400,2),np.round(np.log(result_2_upper['moment_bound_cond'][0])*400,2),np.round(np.log(result_3_lower['moment_bound_cond'][0])*400,2),np.round(np.log(result_3_upper['moment_bound_cond'][0])*400,2)))
print('mid D/P           %s                 %s                  %s' \
      % (np.round(np.log(result_2_min['moment_cond'][1])*400,2),np.round(np.log(result_2_min['moment_bound_cond'][1])*400,2),np.round(np.log(result_3_min['moment_bound_cond'][1])*400,2)))
print('                                  (%s,%s)            (%s,%s)' \
      % (np.round(np.log(result_2_lower['moment_bound_cond'][1])*400,2),np.round(np.log(result_2_upper['moment_bound_cond'][1])*400,2),np.round(np.log(result_3_lower['moment_bound_cond'][1])*400,2),np.round(np.log(result_3_upper['moment_bound_cond'][1])*400,2)))
print('high D/P          %s               %s                   %s' \
      % (np.round(np.log(result_2_min['moment_cond'][2])*400,2),np.round(np.log(result_2_min['moment_bound_cond'][2])*400,2),np.round(np.log(result_3_min['moment_bound_cond'][2])*400,2)))
print('                                  (%s,%s)           (%s,%s)' \
      % (np.round(np.log(result_2_lower['moment_bound_cond'][2])*400,2),np.round(np.log(result_2_upper['moment_bound_cond'][2])*400,2),np.round(np.log(result_3_lower['moment_bound_cond'][2])*400,2),np.round(np.log(result_3_upper['moment_bound_cond'][2])*400,2)))
print('none              %s                %s                  %s' \
      % (np.round(np.log(result_2_min['moment'])*400,2),np.round(np.log(result_2_min['moment_bound'])*400,2),np.round(np.log(result_3_min['moment_bound'])*400,2)))
print('                                  (%s,%s)           (%s,%s)' \
      % (np.round(np.log(result_2_lower['moment_bound'])*400,2),np.round(np.log(result_2_upper['moment_bound'])*400,2),np.round(np.log(result_3_lower['moment_bound'])*400,2),np.round(np.log(result_3_upper['moment_bound'])*400,2)))
print('--------------------------------------------------------------------')
print('note: the numbers in the parentheses impose a quadratic divergence')
print('      constraint that is %s percent higher than the minimum.' % (int(x_min_QD*100-100)))

Table 2: Expected market return
--------------------------------------------------------------------
conditioning    empirical    min divergence imputed    Rw/Rf imputed
                 average         (lower, upper)        (lower,upper)
--------------------------------------------------------------------
low D/P           6.54                3.64                  2.98
                                  (2.87,4.46)           (2.64,3.34)
mid D/P           4.7                 3.78                  2.19
                                  (3.5,4.09)            (1.77,2.66)
high D/P          15.41               6.4                   3.37
                                  (5.93,6.91)           (2.86,3.92)
none              8.94                3.75                  2.84
                                  (2.97,4.53)           (2.43,3.25)
--------------------------------------------------------------------
note: the numbers in the parentheses impose a quadratic divergence
      constraint that is

In [16]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
    
# Find ξs that correspond to 0-25 percent higher min QD
# ξs_lower = []
# ξs_upper = []
# for i in range(0,6):
#     print(i)
#     if i == 0:
#         ξs_lower.append(10)
#         ξs_upper.append(10)
#     else:
#         ξ_lower_temp = solver_1.find_ξ(x_min_QD=(i*5+100)/100,lower=True, tol=1e-7,max_iter=100)
#         ξ_upper_temp = solver_1.find_ξ(x_min_QD=(i*5+100)/100,lower=False,tol=1e-7,max_iter=100)
#         ξs_lower.append(ξ_lower_temp)
#         ξs_upper.append(ξ_upper_temp)

time_start = time.time()
result_1_lower_list = []
result_1_upper_list = []
for i in range(0,6):
    result_1_lower_list.append(solver_1.iterate(ξs_lower[i],lower=True))
    result_1_upper_list.append(solver_1.iterate(ξs_upper[i],lower=False))

def f(percent):
    box_chart(result_1_min,result_1_lower_list[int(percent/5)],result_1_upper_list[int(percent/5)])
    
interact(f, percent=widgets.IntSlider(min=0, max=25, step=5, value=20));
print("Time spent: %s seconds ---" % (round(time.time()-time_start,4)))

interactive(children=(IntSlider(value=20, description='percent', max=25, step=5), Output()), _dom_classes=('wi…

Time spent: 17.6686 seconds ---


In [ ]:
# box_chart(result_1_min,result_1_lower,result_1_upper,True,dpi=None)